In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import re

# File path of the input Excel
file_path = "Kadam Haryana compile report  2024-2025 02052025.xlsx"

# Load the data sheet into pandas DataFrame
data_df = pd.read_excel(file_path, sheet_name='Compile Report')

# Define validation rules in the script as a dictionary
# Format: column_name: list of validations to apply
# Supported validations: "not null", "numeric", "date", "no_special_chars"
validation_rules = {
    "Student's First Name": ["not null", "no_special_chars"],
    "Student's Last Name": ["not null", "no_special_chars"],
    "Student's Age": ["not null", "numeric"],
    "Student's Date of Birth": ["date"],
    "Father's Age": ["not null", "no_special_chars"],
    "Father's Occupation": ["not null", "no_special_chars"],
    "Father's Education": ["not null", "no_special_chars"],
    "Mother's Name": ["not null", "numeric"],
    "Mother's Age": ["not null", "no_special_chars"],
    "Mother's Occupation": ["not null", "no_special_chars"],
    "How long are you planning to stay in this area?": ["not null"],
    "Contact No.": ["not null", "numeric"],
    "House Address": ["not null"],
    "Pincode": ["not null", "numeric"],
    "People living in house": ["not null", "numeric"],
    "Cast": ["not null", "no_special_chars"],
    "Religion": ["not null", "no_special_chars"],
    "Parents' Monthly Income": ["not null"],
    "Parents' Monthly Expenditure": ["not null"],
    "Baseline Math": ["not null", "numeric"],
    "Baseline English": ["not null", "numeric"],
    "Baseline EVS": ["not null", "numeric"],
    "Baseline Hindi": ["not null", "numeric"],
    "Baseline Total": ["not null", "numeric"],
    "Baseline Percentage": ["not null", "numeric"],
    "Grade Test 1": ["not null", "numeric"],
    "Grade Test 2": ["not null", "numeric"],
    "Grade Test 3": ["not null", "numeric"],
    "Grade Test 4": ["not null", "numeric"],
    "Grade Test 5": ["not null", "numeric"],
    "Endline Math": ["not null", "numeric"],
    "Endline English": ["not null", "numeric"],
    "Endline EVS": ["not null", "numeric"],
    "Endline Hindi": ["not null", "numeric"],
    "Endline Total": ["not null", "numeric"],
    "Mainstream Institution Name": ["not null"],
    "Mainstream Institution Address": ["not null"],
    "School DISE Code": ["not null", "numeric"],
    "Mainstream Grade": ["not null"],
    "Mainstream Section": ["not null"],
    "Child SR given by the Institution": ["not null"],
    "State": ["not null"],
    "District": ["not null"],

    # Add your columns and validation rules here as needed Father's Age

}

# Define red fill for errors in Excel
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

# Load workbook and worksheet for applying Excel formatting
wb = load_workbook(file_path)
ws = wb['Compile Report']

# List to hold validation error records
validation_errors = []

# Iterate over the validation_rules dictionary
for col_name, rules in validation_rules.items():
    if col_name not in data_df.columns:
        # Column not in data, skip or optionally log warning
        continue

    # Find the Excel column letter for the column header
    col_letter = None
    for cell in ws[1]:
        if cell.value == col_name:
            col_letter = cell.column_letter
            break
    if col_letter is None:
        # Could not find the column in Excel, skip
        continue

    # Iterate over each value in the column
    for i, value in enumerate(data_df[col_name], start=2):  # Excel rows start at 2 due to header
        for rule in rules:
            has_error = False
            error_reason = ""
            rule_lower = rule.lower()

            if rule_lower == "not null":
                if pd.isnull(value) or (isinstance(value, str) and value.strip() == ""):
                    has_error = True
                    error_reason = "Value is null or empty"
            elif rule_lower == "numeric":
                try:
                    float(value)
                except Exception:
                    has_error = True
                    error_reason = "Value is not numeric"
            elif rule_lower == "date":
                if pd.to_datetime(value, errors='coerce') is pd.NaT:
                    has_error = True
                    error_reason = "Value is not a valid date"
            elif rule_lower == "no_special_chars":
                if not re.match("^[A-Za-z ]*$", str(value)):
                    has_error = True
                    error_reason = "Value contains special characters"

            # Add additional rules here as desired

            if has_error:
                cell_ref = f"{col_letter}{i}"
                ws[cell_ref].fill = red_fill
                validation_errors.append({
                    "Row": i,
                    "Column": col_name,
                    "Cell": cell_ref,
                    "Value": value,
                    "Error": error_reason
                })
                # Once error found for this rule in this cell, no need to check other rules for same cell
                break

# Save the highlighted workbook as new file
wb.save("Validated_Output.xlsx")

# Create a DataFrame for validation report
report_df = pd.DataFrame(validation_errors)

# Save the validation report as a separate Excel file (empty if no errors)
if report_df.empty:
    report_df = pd.DataFrame(columns=["Row", "Column", "Cell", "Value", "Error"])

report_df.to_excel("Validation_Report.xlsx", index=False)

print("Validation complete.")
print("Highlighted data saved as 'Validated_Output.xlsx'.")
print("Validation report saved as 'Validation_Report.xlsx'.")
